# Geemap: Data Export and Local Analysis

While `xee` allows for direct lazy loading, sometimes it is necessary to download data for offline processing or specific legacy workflows. This notebook demonstrates how to use `geemap` to export Earth Engine data to local storage and then analyze it using the standard Python geospatial stack.

In [ ]:
import ee
import geemap
import xarray as xr
import rioxarray
import os
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

## Exporting Data with Geemap

Geemap provides convenient functions to download ImageCollections or Images directly to your local machine as GeoTIFF files.

In [ ]:
# Define an area of interest
roi = ee.Geometry.Point([77.59, 12.97]).buffer(1000).bounds()

# Select an image
image = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterBounds(roi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .first() \
    .select(['SR_B4', 'SR_B3', 'SR_B2'])

# Export to local GeoTIFF
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
if not os.path.exists(out_dir): os.makedirs(out_dir)
filename = os.path.join(out_dir, "landsat_export.tif")

geemap.ee_export_image(image, filename=filename, scale=30, region=roi, file_per_band=False)

## Loading Exported Data with Xarray

Once downloaded, we can use `rioxarray` to load the GeoTIFF into an Xarray dataset for local, lightning-fast analysis.

In [ ]:
# Load the local file
if os.path.exists(filename):
    rds = rioxarray.open_rasterio(filename)
    
    # Convert to Xarray Dataset and rename bands if needed
    print(rds)
    
    # Plotting
    plt.figure(figsize=(10, 8))
    rds.plot.imshow(robust=True)
    plt.title("Locally Loaded Landsat Export")
    plt.show()

## Comparing Direct Loading vs. Export

| Feature | Direct (Xee) | Export (Geemap) |
|---------|--------------|-----------------|
| Latency | High (Network) | Low (Local Disk) |
| Storage | None (Lazy) | Required (Disk Space) |
| Scale | Best for summaries | Best for full-res local work |
| Usage | Real-time Analysis | Repeatable Workflows |